In [ ]:
import json
import matplotlib.pyplot as plt
import networkx as nx
from geopy.distance import geodesic

# Load the tour data
with open("safaris_30_items_enhanced.json", "r", encoding="utf-8") as f:
    tours = json.load(f)

In [ ]:

# Define a dictionary for district coordinates
district_coordinates = {
    "gampaha": (7.15, 79.85),
    "ratnapura": (6.7, 80.4),
    "colombo": (6.9271, 79.8612),
    "kandy": (7.2906, 80.6337),
    "galle": (6.0535, 80.2210),
    "jaffna": (9.6647, 80.0177),
    "anuradhapura": (8.3114, 80.4037),
    "polonnaruwa": (7.9381, 81.0069),
    "trincomalee": (8.5875, 81.2114),
    "batticaloa": (7.7289, 81.7018),
    "kurunegala": (7.4878, 80.3653),
    "nuwara eliya": (6.9497, 80.7891),
    "badulla": (6.9933, 81.0550),
    "monaragala": (6.8732, 81.3592),
    "hambantota": (6.1242, 81.1219)
}


In [ ]:

def parse_coordinate(coord_str):
    """
    Converts coordinate strings like '7.15° N' or '79.85° E' to float values.
    Assumes N and E are positive, S and W are negative.
    """
    if not isinstance(coord_str, str):
        return float(coord_str)  # already numeric

    coord_str = coord_str.strip().replace("°", "")
    if "N" in coord_str or "E" in coord_str:
        return float(coord_str.replace("N", "").replace("E", "").strip())
    elif "S" in coord_str or "W" in coord_str:
        return -float(coord_str.replace("S", "").replace("W", "").strip())
    else:
        return float(coord_str.strip())  # fallback

def find_safaris_near_location(user_text, data):
    """
    Finds and sorts safari tours near a specified district.
    """
    user_text_lower = user_text.lower().strip()
    
    # Get coordinates from the dictionary or use defaults
    user_coords = district_coordinates.get(user_text_lower)
    if user_coords is None:
        print(f"⚠️ Warning: Coordinates for '{user_text}' not found. Using default coordinates for Gampaha (7.15, 79.85).")
        user_coords = (7.15, 79.85)

    results = []

    for item in data:
        features = item.get("extracted_features", {})
        if "latitude" not in features or "longitude" not in features:
            continue  # skip if missing coords

        try:
            lat = parse_coordinate(features["latitude"])
            lon = parse_coordinate(features["longitude"])
        except (ValueError, KeyError):
            continue  # skip on bad format or missing keys

        safari_coords = (lat, lon)
        distance = geodesic(user_coords, safari_coords).km

        if features.get("district", "").lower() == user_text_lower or distance <= 50:
            item["distance_km"] = round(distance, 2)
            results.append(item)

    # Sort results by rating (descending) and then distance (ascending)
    sorted_results = sorted(results, key=lambda x: (-float(x.get("rating", 0)), x.get("distance_km", float('inf'))))
    
    return sorted_results


In [ ]:

def visualize_roadmap_graph(roadmap):
    """
    Visualizes the top safari tours as a directed graph.
    """
    if not roadmap:
        print("🚫 No roadmap to display.")
        return

    G = nx.DiGraph()

    labels = {}
    for idx, item in enumerate(roadmap):
        title = item["title"]
        # Use the first park name found, or 'Unknown' if none
        park = item["extracted_features"]["park_names"][0] if item["extracted_features"].get("park_names") else "Unknown"
        label = f"{idx+1}. {park}\n⭐{item.get('rating', '?')}"
        G.add_node(title)
        labels[title] = label

    # Add edges to form a path connecting the top safaris
    for i in range(len(roadmap) - 1):
        G.add_edge(roadmap[i]["title"], roadmap[i+1]["title"])

    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(14, 8))
    nx.draw_networkx_nodes(G, pos, node_size=2500)
    nx.draw_networkx_edges(G, pos, arrows=True, arrowsize=20, connectionstyle='arc3,rad=0.2')
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=9)

    plt.title("🗺️ Safari Tour Roadmap (Graph View)", fontsize=14)
    plt.axis("off")
    plt.tight_layout()
    plt.show()


In [ ]:

# --- Main execution part ---
user_query = input("Please enter your current district: ")
matching_tours = find_safaris_near_location(user_query, tours)

# Display the top 5 matching tours
roadmap = matching_tours[:5]
visualize_roadmap_graph(roadmap)